In [1]:
# # Initializing Spark
# import findspark
# findspark.init()

# # Importing PySpark related
# from pyspark import SparkContext
# from pyspark.sql import SparkSession
# import collections

In [2]:
# # Importing TensorFlow
# import tensorflow as tf
# import tensorflowonspark as tfos
# from tensorflowonspark import TFCluster # Needs package 'packaging' to be installed manually to run properly

# from tensorflow.keras import layers, models
# import tensorflow.keras as keras

In [3]:
# Importing other packages
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from IPython import display

# Hiding pysoundfile warning generated by librosa
import warnings
warnings.filterwarnings("ignore", message="pysoundfile") 

In [14]:
# Testing loading of audio data
data, sampleRate = librosa.load('./data/cv-valid-train/cv-valid-train/sample-124273.mp3', sr=16000)

In [15]:
display.Audio(data, rate=16000)

In [4]:
text = pd.read_csv('./data/cv-valid-train.csv')

In [5]:
textOnly = pd.DataFrame(text.reindex(text.text.str.len().sort_values().index)).reset_index().drop(['index'], axis=1)

In [6]:
firstTest = textOnly.head(100)

In [11]:
firstTest

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-train/sample-030942.mp3,amen,1,0,NaN,NaN,NaN,NaN
1,cv-valid-train/sample-182350.mp3,amen,2,0,thirties,male,us,NaN
2,cv-valid-train/sample-190328.mp3,amen,1,0,twenties,male,canada,NaN
3,cv-valid-train/sample-142724.mp3,amen,1,0,NaN,NaN,NaN,NaN
4,cv-valid-train/sample-012246.mp3,amen,1,0,NaN,NaN,us,NaN
...,...,...,...,...,...,...,...,...
95,cv-valid-train/sample-174395.mp3,may i see it,1,0,teens,female,african,NaN
96,cv-valid-train/sample-077104.mp3,mind the gap,1,0,fourties,male,england,NaN
97,cv-valid-train/sample-052006.mp3,may i see it,1,0,NaN,NaN,NaN,NaN
98,cv-valid-train/sample-050555.mp3,try it again,1,0,NaN,NaN,NaN,NaN


In [12]:
# Function to load audio data
def loadAudio(filename):
    data, _ = librosa.load(f'./data/cv-valid-train/{filename}', sr=16000)
    bands = getBands(data)
    padded = padAudio(bands, 1000)
    return padded.reshape(1, 1000, 1025)

In [13]:
X = []
for i in tqdm(firstTest.iterrows(), total=firstTest.shape[0]):
    X.append(loadAudio(i[1]['filename']))

In [14]:
X = np.array(X)

In [15]:
textAsList = list(firstTest['text'])
for index, txt in enumerate(textAsList):
    textAsList[index] = list(txt.upper().replace(' ', '_'))
    while len(textAsList[index]) < 200:
        textAsList[index].append(' ')
    # Encode in ASCII
    textAsList[index] = [ord(i) for i in textAsList[index]]

In [16]:
y = np.array(textAsList)

In [17]:
y

array([[65, 77, 69, ..., 32, 32, 32],
       [65, 77, 69, ..., 32, 32, 32],
       [65, 77, 69, ..., 32, 32, 32],
       ...,
       [77, 65, 89, ..., 32, 32, 32],
       [84, 82, 89, ..., 32, 32, 32],
       [65, 78, 68, ..., 32, 32, 32]])

In [ ]:
# Plotting audio wave
plt.figure(figsize=(12, 4))
librosa.display.waveplot(data, sr = sampleRate)

In [ ]:
test1 = data[0:320]

In [ ]:
np.abs([i[0] for i in librosa.stft(data[:320], hop_length = 321)])

In [4]:
def getBands(data):
    return np.array([np.abs([j[0] for j in librosa.stft(data[i*320:(i+1)*320], hop_length = 321)]) for i in range(int(len(data)/320))])
#     for i in range(int(len(data)/320)):
#         test = data[i*320:(i+1)*320]
#         samples.append(np.abs(librosa.stft(test, hop_length = 321)))
        
#     return np.array(samples)

In [ ]:
test1[0]

In [ ]:
bands = getBands(data)

In [ ]:
bands.shape

In [ ]:
librosa.display.specshow(bands[0], y_axis = 'log', cmap='Spectral')

In [5]:
# Function to pad audio arrays to equal length for neural net
def padAudio(track, desired):
#     print(track.shape)
    
    padding = desired - track.shape[0]
    filler = np.array([0.0 for i in range(len(track[0]))])
#     .reshape(-1, 1)
    appendix = np.array([filler for i in range(padding)])
#     print(appendix.shape)
    
    return np.concatenate((track, appendix), axis = 0)

In [ ]:
for i in range(160000 - data.shape[0]):
    data = np.concatenate((data, [0.0]), axis=0)

In [ ]:
display.Audio(data, rate=16000)

In [ ]:
data.shape

In [ ]:
bands.shape

In [ ]:
lel = padAudio(bands, 1000)

In [ ]:
lel.reshape(1, 1000, 1025).shape

In [ ]:
# reshaped = np.array(list(lel.reshape(-1,)))

In [ ]:
# reshaped.shape

In [ ]:
# plt.figure(figsize=(12, 4))
# librosa.display.waveplot(reshaped, sr = sampleRate)

In [ ]:
# display.Audio(reshaped, rate = 44100)

In [ ]:
maxLen = 0
for i in tqdm(range(1000)):
    loopData, _ = librosa.load(f'./data/cv-valid-train/cv-valid-train/sample-{"{:06d}".format(i)}.mp3', res_type = 'kaiser_fast', sr=16000)
    length = getBands(loopData).shape[0]
    if (length > maxLen):
        maxLen = length
        
print(maxLen)

In [ ]:
print("{:06d}".format(100))

In [18]:
model = models.Sequential()
model.add(layers.Input(shape=(1, 1000, 1025)))
model.add(layers.Conv2D(32, kernel_size=[5,5], padding='same', activation='relu', data_format='channels_first'))
model.add(layers.MaxPooling2D(pool_size=[1,5], data_format='channels_first'))
model.add(layers.Conv2D(64, kernel_size=[5,5], padding='same', activation='relu', data_format='channels_first'))
model.add(layers.MaxPooling2D(pool_size=[1,5], data_format='channels_first'))
model.add(layers.Conv2D(128, kernel_size=[5,5], padding='same', activation='relu', data_format='channels_first'))
model.add(layers.MaxPooling2D(pool_size=[1,5], data_format='channels_first'))
model.add(layers.Conv2D(256, kernel_size=[5,5], padding='same', activation='relu', data_format='channels_first'))
model.add(layers.MaxPooling2D(pool_size=[1,5], data_format='channels_first'))
model.add(layers.Reshape((1000,256)))
model.add(layers.LSTM(256, activation='tanh', dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(200, activation='softmax'))

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 1000, 1025)    832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 1000, 205)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 1000, 205)     51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 1000, 41)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 1000, 41)     204928    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 128, 1000, 8)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 1000, 8)      8

In [ ]:
model.fit(X, y, verbose=1)

Train on 100 samples


In [ ]:
model.predict(X)

In [30]:
textOnly['text'].value_counts()

it had a diameter of about thirty yards                      265
most of them were staring quietly at the big table           259
a boy came running towards me                                258
you point to a trail of ants leading into the house          256
the time must have been somewhere around six o'clock         256
                                                            ... 
ship technology has improved                                   1
see the emacs lisp for this at the bottom of the post          1
need to find the tv series called administrative behavior      1
give one out of  to pincher martin saga                        1
whenever i fall asleep i don't dream                           1
Name: text, Length: 6994, dtype: int64

In [31]:
testingSentence = textOnly[textOnly['text'] == 'it had a diameter of about thirty yards']

In [32]:
testingSentence.shape

(265, 8)

In [33]:
pd.DataFrame(text.reindex(testingSentence.text.str.len().sort_values().index)).reset_index().drop(['index'], axis=1)

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-train/sample-073479.mp3,what were you doing in boston,3,0,thirties,male,us,NaN
1,cv-valid-train/sample-076271.mp3,it had a diameter of about thirty yards,4,0,thirties,male,us,NaN
2,cv-valid-train/sample-076275.mp3,he was selling better than ever,5,0,fourties,male,us,NaN
3,cv-valid-train/sample-076293.mp3,then you'll die in the midst of trying to real...,2,0,NaN,NaN,NaN,NaN
4,cv-valid-train/sample-076300.mp3,coming home a party of tourists passed us sing...,2,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
260,cv-valid-train/sample-075038.mp3,there was no way for everybody to know about a...,2,1,NaN,NaN,NaN,NaN
261,cv-valid-train/sample-075046.mp3,it does impose some constraints on command ord...,1,0,NaN,NaN,NaN,NaN
262,cv-valid-train/sample-075047.mp3,maybe tomorrow will be better than yesterday,2,0,fourties,NaN,australia,NaN
263,cv-valid-train/sample-074842.mp3,it could prove fatal for them as well as yourself,1,0,NaN,NaN,NaN,NaN


In [34]:
testingSentence.to_pickle('testingSentence')